In [1]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    run_on_colab = True
else:
    print('Not running on CoLab')
    run_on_colab = False
if run_on_colab:
    from google.colab import drive
    drive.mount('/drive', force_remount=True)

Running on CoLab
Mounted at /drive


In [2]:
!pip install ipython-autotime
%load_ext autotime

time: 213 µs (started: 2021-06-05 16:40:43 +00:00)


In [3]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib
import matplotlib.pyplot as plt

time: 980 ms (started: 2021-06-05 16:40:43 +00:00)


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [9]:
window_length = 5
threshold=0.15
rocs=1
clf = None
scaler = None

time: 1.68 ms (started: 2021-06-05 16:41:54 +00:00)


In [29]:
train_df = pd.read_csv("/drive/My Drive/disertation/train_df_{}_rocs_{}.csv".format(threshold,rocs))
train_df = train_df.set_index("open_time")
train_df.drop(columns=["close"],inplace=True)
train_df

,sma5_diff,sma8_diff,sma13_diff,sma21_diff,ema5_diff,ema8_diff,ema13_diff,ema21_diff,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,ROC_1,STOCHk_14_3_3,STOCHd_14_3_3,CCI_14_0.015,ATRr_14,WILLR_14,AROOND_14,AROONU_14,AROONOSC_14,dema10_diff,BBB_5_2.0,BBU_5_2.0_diff,BBL_5_2.0_diff,BBM_5_2.0_diff,RVI_14,output_class
open_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-08-17 04:33:00,-0.000882,-0.000402,-0.000094,0.000409,-0.000701,-0.000509,-0.000204,0.000135,2.517063,0.710935,1.806128,52.037221,-0.156269,57.813583,64.134419,-1.973131,3.477987,-65.332640,64.285714,78.571429,14.285714,-0.000948,0.670908,-0.004240,0.002475,-0.000882,26.024920,0.0
2017-08-17 04:34:00,0.004152,0.005384,0.006029,0.006682,0.003980,0.004791,0.005539,0.006182,4.527372,2.176995,2.350377,73.578620,0.671042,68.018018,68.150843,285.454223,5.420447,0.000000,57.142857,100.000000,42.857143,0.003748,1.013689,-0.000896,0.009199,0.004152,58.444576,1.0
2017-08-17 04:35:00,0.004072,0.005260,0.006286,0.007118,0.003232,0.004401,0.005491,0.006408,6.349951,3.199659,3.150292,75.152240,0.087278,78.222453,68.018018,212.079230,5.292232,0.000000,50.000000,100.000000,50.000000,0.002882,1.362926,-0.002715,0.010859,0.004072,75.155465,1.0
2017-08-17 04:36:00,0.002876,0.004269,0.005676,0.006687,0.002155,0.003423,0.004707,0.005825,7.705534,3.644193,4.061340,75.152240,0.000000,100.000000,82.080157,156.805529,4.888177,0.000000,42.857143,100.000000,57.142857,0.001626,1.444458,-0.004325,0.010078,0.002876,75.155465,1.0
2017-08-17 04:37:00,0.001681,0.003138,0.005066,0.006256,0.001437,0.002663,0.004034,0.005296,8.679787,3.694758,4.985030,75.152240,0.000000,100.000000,92.740818,120.704989,4.516799,0.000000,35.714286,100.000000,64.285714,0.000732,1.319255,-0.004904,0.008266,0.001681,75.155465,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-29 15:07:00,0.000561,0.000571,0.000239,0.000047,0.000337,0.000346,0.000218,-0.000055,-3.690980,0.308423,-3.999402,47.534596,0.014855,50.769231,35.601140,21.098279,6.758089,-40.205128,64.285714,7.142857,-57.142857,0.000531,0.207261,-0.000475,0.001596,0.000561,56.326805,1.0
2018-07-29 15:08:00,0.000527,0.000675,0.000470,0.000239,0.000352,0.000418,0.000351,0.000124,-3.128508,0.696716,-3.825224,49.653041,0.019113,60.592056,48.294218,59.137727,6.430368,-31.557165,57.142857,0.000000,-57.142857,0.000517,0.223438,-0.000590,0.001644,0.000527,61.932170,1.0
2018-07-29 15:09:00,0.000366,0.000728,0.000598,0.000375,0.000337,0.000444,0.000432,0.000252,-2.551659,1.018851,-3.570511,51.350797,0.015336,67.862907,59.741398,65.158862,6.421771,-24.648986,50.000000,0.000000,-50.000000,0.000465,0.129509,-0.000281,0.001013,0.000366,66.525812,1.0


time: 8.9 s (started: 2021-06-05 17:26:35 +00:00)


In [30]:
train_df.index = pd.to_datetime(train_df.index)

time: 136 ms (started: 2021-06-05 17:26:53 +00:00)


In [31]:
columns = []
for window_idx in range(window_length):
    for column in train_df.columns:
        if column == "output_class":
            continue
        columns.append("{}___{}".format(column, window_idx))
len(columns)

135

time: 9.32 ms (started: 2021-06-05 17:26:54 +00:00)


In [32]:
data = train_df.to_numpy()
X = data[:,:-1]
Y = data[:,-1]

X.shape, Y.shape

((498879, 27), (498879,))

time: 6.88 ms (started: 2021-06-05 17:26:55 +00:00)


In [33]:
def extend_dataset_with_window_length(X, Y, window_length=window_length):
    new_x = []
    for i in range(len(X) - window_length+1):
        lst = []
        for j in range(i, i+window_length):
            lst.extend(X[j])
        new_x.append(lst)
    return np.array(new_x), Y[window_length-1:]

time: 6.47 ms (started: 2021-06-05 17:26:56 +00:00)


In [34]:
X, Y = extend_dataset_with_window_length(X,Y)
X.shape, Y.shape

((498875, 135), (498875,))

time: 38.4 s (started: 2021-06-05 17:27:01 +00:00)


In [36]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

time: 919 ms (started: 2021-06-05 17:27:46 +00:00)


In [78]:
# from sklearn.preprocessing import StandardScaler
# standard_scaler = StandardScaler()
# X= standard_scaler.fit_transform(X)

time: 2.58 s (started: 2021-06-04 22:14:56 +00:00)


In [13]:
w# scaler_path = "/drive/My Drive/disertation/scaler_model.save"
# if scaler:
#     joblib.dump(scaler, scaler_path)
# else:
#     scaler = joblib.load(scaler_path)

time: 3.73 ms (started: 2021-06-04 19:43:17 +00:00)


In [37]:
a,b = np.unique(Y, return_counts=True)
print("buy hold sell %")
print(b)
print(b / sum(b))

buy hold sell %
[ 60627 377448  60800]
[0.12152744 0.75659835 0.12187422]
time: 23.9 ms (started: 2021-06-05 17:27:46 +00:00)


In [38]:
def get_subsample_same_distribution(X,Y, p=0.1):
    ys = pd.Series(Y)
    vs = ys.value_counts()
    original_indexes = []
    for idx in vs.index:
        required = int(p * vs[idx])
        print(required)
        original_indexes.extend(ys[ys==idx].sample(n=required, replace=False).index)
    return X[original_indexes], Y[original_indexes]

time: 7.33 ms (started: 2021-06-05 17:27:46 +00:00)


In [39]:
# X,Y = get_subsample_same_distribution(X,Y)
X.shape, Y.shape

((498875, 135), (498875,))

time: 4.69 ms (started: 2021-06-05 17:27:46 +00:00)


In [40]:
max_depth = 20
clf = DecisionTreeClassifier(max_depth=max_depth, class_weight="balanced")
clf.fit(X, Y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=20, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

time: 2min 49s (started: 2021-06-05 17:27:47 +00:00)


In [41]:
d = {"feature":columns, "importance": clf.feature_importances_}
pd.DataFrame(d).sort_values("importance", ascending=False).head(20)

,feature,importance
130,BBB_5_2.0___4,0.112526
22,BBB_5_2.0___0,0.034635
124,ATRr_14___4,0.033782
120,ROC_1___4,0.033663
112,ema5_diff___4,0.025806
125,WILLR_14___4,0.021290
16,ATRr_14___0,0.020445
123,CCI_14_0.015___4,0.013319
134,RVI_14___4,0.012511
132,BBL_5_2.0_diff___4,0.012047


time: 24.1 ms (started: 2021-06-05 17:30:50 +00:00)


In [42]:
print(clf.score(X, Y))
Y_train_predicted = clf.predict(X)

print(np.array(np.unique(Y, return_counts=True)).T)

a,b = np.unique(Y, return_counts=True)
print("buy hold sell %")
print(b / sum(b))

print(np.array(np.unique(Y_train_predicted, return_counts=True)).T)

a,b = np.unique(Y_train_predicted, return_counts=True)
print("buy hold sell %")
print(b / sum(b))

0.7945457278877475
[[0.00000e+00 6.06270e+04]
 [1.00000e+00 3.77448e+05]
 [2.00000e+00 6.08000e+04]]
buy hold sell %
[0.12152744 0.75659835 0.12187422]
[[0.00000e+00 9.90220e+04]
 [1.00000e+00 2.98331e+05]
 [2.00000e+00 1.01522e+05]]
buy hold sell %
[0.1984906  0.59800752 0.20350188]
time: 704 ms (started: 2021-06-05 17:31:00 +00:00)


In [43]:
Y_train_predicted = clf.predict(X)
print(metrics.confusion_matrix(Y, Y_train_predicted))

# Print the precision and recall, among other metrics
print(metrics.classification_report(Y, Y_train_predicted, digits=3))

[[ 52874   4024   3729]
 [ 42534 290313  44601]
 [  3614   3994  53192]]
              precision    recall  f1-score   support

         0.0      0.534     0.872     0.662     60627
         1.0      0.973     0.769     0.859    377448
         2.0      0.524     0.875     0.655     60800

    accuracy                          0.795    498875
   macro avg      0.677     0.839     0.726    498875
weighted avg      0.865     0.795     0.810    498875

time: 2.02 s (started: 2021-06-05 17:31:09 +00:00)


In [44]:
clf_file = "/drive/My Drive/disertation/decision_tree_rocs_{}_balanced_at_train_unscaled_depth_{}.pickle".format(rocs, max_depth)
if clf:
    pickle.dump(clf, open(clf_file, 'wb'))
else:
    clf = pickle.load(open(clf_file, "rb"))

time: 72.6 ms (started: 2021-06-05 17:31:20 +00:00)


In [45]:
from sklearn import tree
text_representation = tree.export_text(clf, feature_names=columns)
print(text_representation)

|--- BBB_5_2.0___4 <= 0.02
|   |--- BBB_5_2.0___0 <= 0.02
|   |   |--- ATRr_14___0 <= 0.02
|   |   |   |--- BBB_5_2.0___1 <= 0.00
|   |   |   |   |--- WILLR_14___4 <= 0.56
|   |   |   |   |   |--- ATRr_14___4 <= 0.01
|   |   |   |   |   |   |--- dema10_diff___4 <= 0.48
|   |   |   |   |   |   |   |--- sma13_diff___4 <= 0.44
|   |   |   |   |   |   |   |   |--- BBU_5_2.0_diff___4 <= 0.99
|   |   |   |   |   |   |   |   |   |--- MACDh_12_26_9___0 <= 0.41
|   |   |   |   |   |   |   |   |   |   |--- RSI_14___0 <= 0.67
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |   |--- RSI_14___0 >  0.67
|   |   |   |   |   |   |   |   |   |   |   |--- class: 2.0
|   |   |   |   |   |   |   |   |   |--- MACDh_12_26_9___0 >  0.41
|   |   |   |   |   |   |   |   |   |   |--- STOCHk_14_3_3___1 <= 0.33
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |   |--- STOCHk_14_3_3___1 >  0.33
|   |   |

In [52]:
tune_df = pd.read_csv("/drive/My Drive/disertation/tune_df_{}_rocs_{}.csv".format(threshold,rocs))
tune_df = tune_df.set_index("open_time")

tune_df.drop(columns="close",inplace=True)

data = tune_df.to_numpy()
X_tune = data[:,:-1]
Y_tune = data[:,-1]

X_tune, Y_tune = extend_dataset_with_window_length(X_tune, Y_tune)

time: 46.6 s (started: 2021-06-05 17:35:28 +00:00)


In [53]:
X_tune = scaler.transform(X_tune)

time: 2.09 s (started: 2021-06-05 17:37:27 +00:00)


In [54]:
a,b = np.unique(Y_tune, return_counts=True)
print("buy hold sell %")
print(b / sum(b))
Y_tune_predicted = clf.predict(X_tune)
print(metrics.confusion_matrix(Y_tune, Y_tune_predicted))

print(metrics.classification_report(Y_tune, Y_tune_predicted, digits=3))


buy hold sell %
[0.03926218 0.9223085  0.03842931]
[[  8901  21292   8981]
 [ 46476 820101  53660]
 [  8160  20953   9230]]
              precision    recall  f1-score   support

         0.0      0.140     0.227     0.173     39174
         1.0      0.951     0.891     0.920    920237
         2.0      0.128     0.241     0.167     38343

    accuracy                          0.840    997754
   macro avg      0.407     0.453     0.420    997754
weighted avg      0.888     0.840     0.862    997754

time: 3.82 s (started: 2021-06-05 17:37:31 +00:00)


In [55]:
# clear ram
del tune_df
del X_tune
del Y_tune

time: 39.3 ms (started: 2021-06-05 17:38:28 +00:00)


In [56]:
test_df = pd.read_csv("/drive/My Drive/disertation/test_df_{}_rocs_{}.csv".format(threshold,rocs))
test_df = test_df.set_index("open_time")
test_df.drop(columns="close", inplace=True)

data = test_df.to_numpy()
X_test = data[:,:-1]
Y_test = data[:,-1]

X_test, Y_test = extend_dataset_with_window_length(X_test, Y_test)

time: 35 s (started: 2021-06-05 17:38:46 +00:00)


In [57]:
X_test = scaler.transform(X_test)

time: 379 ms (started: 2021-06-05 17:39:21 +00:00)


In [58]:
a,b = np.unique(Y_test, return_counts=True)
print("buy hold sell %")
print(b / sum(b))
Y_test_predicted = clf.predict(X_test)
print(metrics.confusion_matrix(Y_test, Y_test_predicted))

print(metrics.classification_report(Y_test, Y_test_predicted, digits=3))

buy hold sell %
[0.06405412 0.87312052 0.06282536]
[[  9866  12465   9624]
 [ 62081 309650  63847]
 [  9588  12203   9551]]
              precision    recall  f1-score   support

         0.0      0.121     0.309     0.174     31955
         1.0      0.926     0.711     0.804    435578
         2.0      0.115     0.305     0.167     31342

    accuracy                          0.660    498875
   macro avg      0.387     0.441     0.382    498875
weighted avg      0.824     0.660     0.724    498875

time: 2.01 s (started: 2021-06-05 17:39:22 +00:00)
